# Synthetic car images

In [1]:
# %load_ext autoreload
# %autoreload 2

In [2]:
import sys
sys.path.append('../')

from utils import repeat_n_times, get_latex_table, hsv_to_name
from IPython.display import display

import torch
import numpy as np
from glob import glob
import pandas as pd
import colorsys
import webcolors


In [3]:
# Hyperparameters
num_repetitions = 5
include_models = ["ae"]

# Creating embeddings

In [4]:
# image_paths = glob("../data/synthetic/cars/*.png")
# image_ids = [int(p.split("/")[-1].replace(".png", "")) for p in image_paths]

# create_image_embeddings(image_paths, image_ids, "../data/synthetic/image_features.pt")

In [5]:
# load image features and labels
image_features = torch.load("../data/synthetic/image_features.pt")
df = pd.read_csv("../data/synthetic/cars.csv")

We only want to use some images for testing, since it may else consume too much memory.

In [6]:
num_images = 1000
image_features = {i: item for i, item in image_features.items() if i <= num_images}
df = df[df["frame"] <= num_images]

In [7]:
image_features = [image_features[i] for i in range(1, num_images + 1)]
image_features = torch.tensor(np.stack(image_features, axis=0))

In [8]:
df.sort_values(by="frame", inplace=True)

# Experiments

## Car Model

In [9]:
# # Scrapes all car model names from the kbb page
# car_models = pd.read_html("https://www.kbb.com/car-make-model-list/new/view-all/make/")
# all_models = car_models[0]["Make"] + " " + car_models[0][".css-1mzj64w-ContentWrapper{margin:10px 24px 10px 0px;}Model"]
# all_models = all_models.dropna().tolist()
# all_models = list(set(all_models))

In [10]:
all_models = torch.load("../data/cars196/all_models_scraped_kbb_2022-08-17.pt")

In [11]:
labels = df["model"].tolist()
texts = [f"a photo of a {m}" for m in all_models]

means, stds = repeat_n_times(num_repetitions, labels, image_features, texts=texts, include_models=include_models)

Creating text features
Run 1
Getting Autoencoder performance


Loss (6937): 8.633151054382324: : 6937it [10:20, 11.19it/s] 


Finished optimization. Best loss (8.495588302612305) achieved after 6837 iterations.
Run 2
Getting Autoencoder performance


Loss (3692): 15.098151206970215: : 3692it [04:05, 15.02it/s]


Finished optimization. Best loss (11.002554893493652) achieved after 3592 iterations.
Run 3
Getting Autoencoder performance


...

Finished optimization. Best loss (8.750614166259766) achieved after 4996 iterations.
Run 4
Getting Autoencoder performance


Loss (5393): 2636094.0: : 5393it [05:32, 16.21it/s]         


Finished optimization. Best loss (8.469369888305664) achieved after 5293 iterations.
Run 5
Getting Autoencoder performance


Loss (8038): 6.151677131652832: : 8038it [11:03, 12.11it/s] 


Finished optimization. Best loss (5.14424991607666) achieved after 7938 iterations.


In [12]:
display(means)
display(stds)

,AE
AMI,0.538841
NMI,0.542116
mean_average_precision,0.550679
mean_average_precision_at_r,0.394928
mean_reciprocal_rank,0.928347
precision_at_1,0.886600
r_precision,0.516815


,AE
AMI,0.059519
NMI,0.059111
mean_average_precision,0.040981
mean_average_precision_at_r,0.043551
mean_reciprocal_rank,0.024122
precision_at_1,0.036357
r_precision,0.035534


In [13]:
print(get_latex_table(means, stds, "Synthetic Data, Car Model", "fig:synthetic_data_car_model"))

\begin{table}
\centering
\caption{Synthetic Data, Car Model}
\label{fig:synthetic_data_car_model}
\begin{tabular}{@{}rc@{}}
\toprule
{} &                 AE \\
\midrule
\textbf{AMI   } &  0.539 $\pm$ 0.060 \\
\textbf{NMI   } &  0.542 $\pm$ 0.059 \\
\textbf{MAP   } &  0.551 $\pm$ 0.041 \\
\textbf{MAP@R } &  0.395 $\pm$ 0.044 \\
\textbf{MRR   } &  0.928 $\pm$ 0.024 \\
\textbf{Prec@1} &  0.887 $\pm$ 0.036 \\
\textbf{R-Prec} &  0.517 $\pm$ 0.036 \\
\bottomrule
\end{tabular}
\end{table}



/Users/konstantinkobs/code/2022-dml-clip/experiments/../utils.py:215: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  return final_results.to_latex(bold_rows=True, escape=False, caption=caption, label=label, column_format=columns_format)


In [14]:
print("### Mean")
print(means.to_markdown())
print("### Std")
print(stds.to_markdown())

### Mean
|                             |       AE |
|:----------------------------|---------:|
| AMI                         | 0.538841 |
| NMI                         | 0.542116 |
| mean_average_precision      | 0.550679 |
| mean_average_precision_at_r | 0.394928 |
| mean_reciprocal_rank        | 0.928347 |
| precision_at_1              | 0.8866   |
| r_precision                 | 0.516815 |
### Std
|                             |        AE |
|:----------------------------|----------:|
| AMI                         | 0.0595186 |
| NMI                         | 0.0591107 |
| mean_average_precision      | 0.0409807 |
| mean_average_precision_at_r | 0.0435515 |
| mean_reciprocal_rank        | 0.024122  |
| precision_at_1              | 0.0363571 |
| r_precision                 | 0.0355335 |


## Car Color

In [15]:
df["color_name"] = df.apply(hsv_to_name, axis=1, hue_column="color_hue", sat_column="color_sat", val_column="color_val")

In [16]:
labels = df["color_name"].tolist()
texts = [f"a {c} car" for c in webcolors.CSS21_HEX_TO_NAMES.values()]

means, stds = repeat_n_times(num_repetitions, labels, image_features, texts=texts, include_models=include_models)

Creating text features
Run 1
Getting Autoencoder performance


Loss (561): 0.0003136740706395358: : 561it [00:09, 56.88it/s] 


Finished optimization. Best loss (1.5156044810282765e-06) achieved after 461 iterations.
Run 2
Getting Autoencoder performance


Loss (828): 0.4865231513977051: : 828it [00:16, 51.13it/s]  


Finished optimization. Best loss (0.009872105903923512) achieved after 728 iterations.
Run 3
Getting Autoencoder performance


...

Finished optimization. Best loss (2.320247176612611e-07) achieved after 847 iterations.
Run 4
Getting Autoencoder performance


Loss (804): 0.00014633165847044438: : 804it [00:16, 50.03it/s]


Finished optimization. Best loss (1.7169277555240114e-07) achieved after 704 iterations.
Run 5
Getting Autoencoder performance


Loss (1386): 0.0002601528831291944: : 1386it [00:29, 46.46it/s] 


Finished optimization. Best loss (1.9168930975865806e-06) achieved after 1286 iterations.


In [17]:
display(means)
display(stds)

,AE
AMI,0.155682
NMI,0.207031
mean_average_precision,0.235520
mean_average_precision_at_r,0.086398
mean_reciprocal_rank,0.477248
precision_at_1,0.302200
r_precision,0.237953


,AE
AMI,0.015837
NMI,0.014828
mean_average_precision,0.004525
mean_average_precision_at_r,0.003908
mean_reciprocal_rank,0.010523
precision_at_1,0.012998
r_precision,0.004387


In [18]:
print(get_latex_table(means, stds, "Synthetic Data, Car Color", "fig:synthetic_data_car_color"))

\begin{table}
\centering
\caption{Synthetic Data, Car Color}
\label{fig:synthetic_data_car_color}
\begin{tabular}{@{}rc@{}}
\toprule
{} &                 AE \\
\midrule
\textbf{AMI   } &  0.156 $\pm$ 0.016 \\
\textbf{NMI   } &  0.207 $\pm$ 0.015 \\
\textbf{MAP   } &  0.236 $\pm$ 0.005 \\
\textbf{MAP@R } &  0.086 $\pm$ 0.004 \\
\textbf{MRR   } &  0.477 $\pm$ 0.011 \\
\textbf{Prec@1} &  0.302 $\pm$ 0.013 \\
\textbf{R-Prec} &  0.238 $\pm$ 0.004 \\
\bottomrule
\end{tabular}
\end{table}



/Users/konstantinkobs/code/2022-dml-clip/experiments/../utils.py:215: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  return final_results.to_latex(bold_rows=True, escape=False, caption=caption, label=label, column_format=columns_format)


hidden size 1024:
```
\begin{table}
\centering
\caption{Synthetic Data, Car Color}
\label{fig:synthetic_data_car_color}
\begin{tabular}{@{}rc@{}}
\toprule
{} &                 AE \\
\midrule
\textbf{AMI   } &  0.108 $\pm$ 0.016 \\
\textbf{NMI   } &  0.162 $\pm$ 0.015 \\
\textbf{MAP   } &  0.217 $\pm$ 0.005 \\
\textbf{MAP@R } &  0.074 $\pm$ 0.003 \\
\textbf{MRR   } &  0.457 $\pm$ 0.006 \\
\textbf{Prec@1} &  0.281 $\pm$ 0.004 \\
\textbf{R-Prec} &  0.221 $\pm$ 0.005 \\
\bottomrule
\end{tabular}
\end{table}
```

hidden size 512:
```
\begin{table}
\centering
\caption{Synthetic Data, Car Color}
\label{fig:synthetic_data_car_color}
\begin{tabular}{@{}rc@{}}
\toprule
{} &                 AE \\
\midrule
\textbf{AMI   } &  0.118 $\pm$ 0.004 \\
\textbf{NMI   } &  0.172 $\pm$ 0.004 \\
\textbf{MAP   } &  0.221 $\pm$ 0.003 \\
\textbf{MAP@R } &  0.075 $\pm$ 0.005 \\
\textbf{MRR   } &  0.453 $\pm$ 0.010 \\
\textbf{Prec@1} &  0.278 $\pm$ 0.016 \\
\textbf{R-Prec} &  0.222 $\pm$ 0.004 \\
\bottomrule
\end{tabular}
\end{table}
```

In [19]:
print("### Mean")
print(means.to_markdown())
print("### Std")
print(stds.to_markdown())

### Mean
|                             |        AE |
|:----------------------------|----------:|
| AMI                         | 0.155682  |
| NMI                         | 0.207031  |
| mean_average_precision      | 0.23552   |
| mean_average_precision_at_r | 0.0863975 |
| mean_reciprocal_rank        | 0.477248  |
| precision_at_1              | 0.3022    |
| r_precision                 | 0.237953  |
### Std
|                             |         AE |
|:----------------------------|-----------:|
| AMI                         | 0.0158365  |
| NMI                         | 0.0148279  |
| mean_average_precision      | 0.00452452 |
| mean_average_precision_at_r | 0.00390814 |
| mean_reciprocal_rank        | 0.0105232  |
| precision_at_1              | 0.0129985  |
| r_precision                 | 0.00438653 |


## Background Color

In [20]:
df["bg_color_name"] = df.apply(hsv_to_name, axis=1, hue_column="bg_color_hue", sat_column="bg_color_sat", val_column="bg_color_val")

In [21]:
labels = df["bg_color_name"].tolist()
texts = [f"a car in front of a {c} background" for c in webcolors.CSS21_HEX_TO_NAMES.values()]

means, stds = repeat_n_times(num_repetitions, labels, image_features, texts=texts, include_models=include_models)

Creating text features
Run 1
Getting Autoencoder performance


Loss (3799): 0.00010371192183811218: : 3799it [01:16, 49.41it/s]


Finished optimization. Best loss (2.9636879844474606e-05) achieved after 3699 iterations.
Run 2
Getting Autoencoder performance


Loss (4034): 0.0010588493896648288: : 4034it [01:12, 55.74it/s] 


Finished optimization. Best loss (1.701995643088594e-06) achieved after 3934 iterations.
Run 3
Getting Autoencoder performance


...

Finished optimization. Best loss (0.006480957381427288) achieved after 3664 iterations.
Run 4
Getting Autoencoder performance


Loss (3260): 0.0011527745518833399: : 3260it [01:01, 52.60it/s] 


Finished optimization. Best loss (2.988586174978991e-06) achieved after 3160 iterations.
Run 5
Getting Autoencoder performance


Loss (4075): 22106.28515625: : 4075it [01:14, 54.43it/s]     


Finished optimization. Best loss (0.288898229598999) achieved after 3975 iterations.


In [22]:
display(means)
display(stds)

,AE
AMI,0.047936
NMI,0.105576
mean_average_precision,0.200084
mean_average_precision_at_r,0.060516
mean_reciprocal_rank,0.391471
precision_at_1,0.216216
r_precision,0.197477


,AE
AMI,0.017710
NMI,0.016602
mean_average_precision,0.004458
mean_average_precision_at_r,0.002200
mean_reciprocal_rank,0.013418
precision_at_1,0.012630
r_precision,0.004020


In [23]:
print(get_latex_table(means, stds, "Synthetic Data, Background Color", "fig:synthetic_data_background_color"))

\begin{table}
\centering
\caption{Synthetic Data, Background Color}
\label{fig:synthetic_data_background_color}
\begin{tabular}{@{}rc@{}}
\toprule
{} &                 AE \\
\midrule
\textbf{AMI   } &  0.048 $\pm$ 0.018 \\
\textbf{NMI   } &  0.106 $\pm$ 0.017 \\
\textbf{MAP   } &  0.200 $\pm$ 0.004 \\
\textbf{MAP@R } &  0.061 $\pm$ 0.002 \\
\textbf{MRR   } &  0.391 $\pm$ 0.013 \\
\textbf{Prec@1} &  0.216 $\pm$ 0.013 \\
\textbf{R-Prec} &  0.197 $\pm$ 0.004 \\
\bottomrule
\end{tabular}
\end{table}



/Users/konstantinkobs/code/2022-dml-clip/experiments/../utils.py:215: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  return final_results.to_latex(bold_rows=True, escape=False, caption=caption, label=label, column_format=columns_format)


hidden size 1024:
```
\begin{table}
\centering
\caption{Synthetic Data, Background Color}
\label{fig:synthetic_data_background_color}
\begin{tabular}{@{}rc@{}}
\toprule
{} &                 AE \\
\midrule
\textbf{AMI   } &  0.047 $\pm$ 0.008 \\
\textbf{NMI   } &  0.105 $\pm$ 0.007 \\
\textbf{MAP   } &  0.203 $\pm$ 0.004 \\
\textbf{MAP@R } &  0.062 $\pm$ 0.003 \\
\textbf{MRR   } &  0.411 $\pm$ 0.007 \\
\textbf{Prec@1} &  0.238 $\pm$ 0.010 \\
\textbf{R-Prec} &  0.200 $\pm$ 0.005 \\
\bottomrule
\end{tabular}
\end{table}
```


hidden size 512:
```
\begin{table}
\centering
\caption{Synthetic Data, Background Color}
\label{fig:synthetic_data_background_color}
\begin{tabular}{@{}rc@{}}
\toprule
{} &                 AE \\
\midrule
\textbf{AMI   } &  0.048 $\pm$ 0.007 \\
\textbf{NMI   } &  0.106 $\pm$ 0.006 \\
\textbf{MAP   } &  0.203 $\pm$ 0.004 \\
\textbf{MAP@R } &  0.063 $\pm$ 0.003 \\
\textbf{MRR   } &  0.412 $\pm$ 0.014 \\
\textbf{Prec@1} &  0.240 $\pm$ 0.012 \\
\textbf{R-Prec} &  0.202 $\pm$ 0.006 \\
\bottomrule
\end{tabular}
\end{table}
```

In [24]:
print("### Mean")
print(means.to_markdown())
print("### Std")
print(stds.to_markdown())

### Mean
|                             |        AE |
|:----------------------------|----------:|
| AMI                         | 0.0479363 |
| NMI                         | 0.105576  |
| mean_average_precision      | 0.200084  |
| mean_average_precision_at_r | 0.0605161 |
| mean_reciprocal_rank        | 0.391471  |
| precision_at_1              | 0.216216  |
| r_precision                 | 0.197477  |
### Std
|                             |         AE |
|:----------------------------|-----------:|
| AMI                         | 0.0177101  |
| NMI                         | 0.0166017  |
| mean_average_precision      | 0.0044584  |
| mean_average_precision_at_r | 0.00219983 |
| mean_reciprocal_rank        | 0.0134176  |
| precision_at_1              | 0.0126301  |
| r_precision                 | 0.0040198  |
